# Code Snippets
This notebook walks through the basics of how to use the AI21 Client (v2.0.1); giving a few code snippets you can use in your own work.

## Instantiate the AI21 client with supressing most output
This will be the first step that you use when using AI21 models. Run this before executing the snippets below.
The AI21 client will produce many logs the standard output. If you wish to supress these, you can use:


In [4]:
from ai21 import AI21Client

client = AI21Client() #uses the environment variable 
#If you wish to pass your api key as a string, you can use the following code
#client = AI21Client(api_key='my_api_key')
import json
import os
os.environ["AI21_LOG_LEVEL"] = "DEBUG"

from ai21 import AI21Client
client = AI21Client()

## Call the Summarize Task Specific Model

AI21 Task specific models (TSM) are foundaiton models that perform specific tasks well. In this case, we will call the AI21 Summarize TSM via the AI21 client.

In [12]:
article='''
The summarize Task Specific model is designed to read in the text; and create a summary
of it that is high accuracy with low hallucination rates. The summary will be a short form summary; and is
designed to be clear and consice.
'''
response = client.summarize.create(
  source=article,
  source_type="TEXT" 
)

summary=response.summary
print(summary)

The summarize Task Specific model is designed to read text and create a short form summary.


## Call Jurassic 2
You can also call Jurassic-2 from the AI21 client

In [ ]:

prompt="write 1 paragraph about the financial industry"
response = client.completion.create(
    model="j2-ultra",  # You can choose from various models like j2-light, j2-mid, j2-ultra
    prompt=prompt,
    max_tokens=300,
    temperature=0.7
)

#get the actual text from the response
generated_text = response.completions[0].data.text
print(generated_text)

## Call Contextual Answers

In [5]:
article='''
AnyCompany is a multinational corporation that offers a wide range of products and services. The company's quarterly
earnings for last year were quite impressive, with a net income of over $2 billion and a profit increase of nearly 20% 
from the previous year. Despite the challenges posed by the pandemic, AnyCompany managed to maintain a strong financial 
performance, thanks to the efforts of its dedicated employees and management team.
'''
response = client.answer.create(
  context=article,
  question="What were the quarterly earnings?" 
)

answer=response.answer
print(answer)

The company's quarterly earnings for last year were quite impressive, with a net income of over $2 billion and a profit increase of nearly 20% from the previous year.


In [6]:
print(response)

AnswerResponse(id='41adb774-17ce-4d67-3607-dcda8ed3475b', answer_in_context=True, answer="The company's quarterly earnings for last year were quite impressive, with a net income of over $2 billion and a profit increase of nearly 20% from the previous year.")


## Create a contextual answers endpoint in Amazon SageMaker


In [ ]:
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3
from ai21 import AI21SageMakerClient
import ai21
model_package_arn = ai21.SageMaker.get_model_package_arn(model_name="contextual-answers", region=region)
role = get_execution_role()
sagemaker_session = sage.Session()
endpoint_name = "contextual-answers"

content_type = "application/json"

real_time_inference_instance_type = (
#    "ml.p4d.24xlarge"    # Recommended instance
#     "ml.g5.48xlarge"    # Cheaper and faster - recommended for relatively short inputs/outputs
#      "ml.g5.12xlarge"    # Even Cheaper and faster - up to 10K characters
      "ml.g4dn.12xlarge"  #cheapest instance  
)

# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name, 
    model_data_download_timeout=3600,
    container_startup_health_check_timeout=600
)

## Call Contextual Answers from Amazon SageMaker
call contextual answers answers from an existing endpoint

In [16]:
from ai21 import AI21SageMakerClient
import boto3
boto_session = boto3.Session(region_name="us-east-1")

article='''
AnyCompany is a multinational corporation that offers a wide range of products and services. The company's quarterly
earnings for last year were quite impressive, with a net income of over $2 billion and a profit increase of nearly 20% 
from the previous year. Despite the challenges posed by the pandemic, AnyCompany managed to maintain a strong financial 
performance, thanks to the efforts of its dedicated employees and management team.
'''
client = AI21SageMakerClient(
    endpoint_name="Endpoint-AI21-Contextual-Answers-1",
)
response = client.answer.create(
  session=boto_session,
  context=article,
  question="What were the quarterly earnings?",
)

print(response)
#answer=response.answer
#print(answer)

AnswerResponse(id='3dc1bd68-765d-4b75-ae4e-fbe887a38ddd', answer_in_context=True, answer="The company's quarterly earnings for last year were quite impressive, with a net income of over $2 billion and a profit increase of nearly 20% from the previous year.")


## Serialize AI21 model responses to JSON
You can save ai21 model responses to json using the to_json() method

In [7]:
response.to_json()

'{"id": "41adb774-17ce-4d67-3607-dcda8ed3475b", "answerInContext": true, "answer": "The company\'s quarterly earnings for last year were quite impressive, with a net income of over $2 billion and a profit increase of nearly 20% from the previous year."}'

## Call AI21 summarize Task Specific Model

In [1]:
role="arn:aws:iam::774836287727:role/service-role/AmazonSageMaker-ExecutionRole-20231205T124436"

In [ ]:
# Expanding the '~' in the output file path
output_file = os.path.expanduser(output_file)

file_details = []

for directory in directories:
    # Expanding the '~' in each directory path
    directory = os.path.expanduser(directory)

    for root, dirs, files in os.walk(directory):
        # Exclude hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        

In [5]:
from ai21 import AI21SageMakerClient
import boto3
boto_session = boto3.Session(region_name="us-east-1")

article='''
The error affected a number of international flights leaving the terminal on Wednesday, with some airlines urging passengers to travel only with hand luggage.
Virgin Atlantic said all airlines flying out of the terminal had been affected.
Passengers have been warned it may be days before they are reunited with luggage.
An airport spokesperson apologised and said the fault had now been fixed.
Virgin Atlantic said it would ensure all bags were sent out as soon as possible.
It added customers should retain receipts for anything they had bought and make a claim to be reimbursed.
Passengers, who were informed by e-mail of the problem, took to social media to vent their frustrations.
One branded the situation "ludicrous" and said he was only told 12 hours before his flight.
The airport said it could not confirm what the problem was, what had caused it or how many people had been affected.
'''
client = AI21SageMakerClient(
    endpoint_name="summarize",
)
response = client.summarize.create(
  session=boto_session,
  source=article,
  source_type="TEXT",
)

print(response)

SummarizeResponse(id='6c6f2d1d-a5c3-4907-9157-d10568a75d3d', summary='Passengers were warned it may be days before they are reunited with luggage, and were informed by e-mail of the problem. They took to social media to vent their frustrations.')


## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 